In [1]:
import cv2
import numpy as np
import os
import pandas
import sys
import keras
import pytesseract
from matplotlib import pyplot as plt
from shutil import copyfile
from PIL import Image, ImageChops
import PIL.ExifTags
import imutils
from random import randint
from skimage import transform
import random

Using TensorFlow backend.


In [2]:
def flip_image(origdir,flippeddir,file,i):
    image = cv2.imread(origdir + "/" + file,0)
    # Try to randomly flip between horizontal and vertical flips.
    #Numbers favor horizontal
    integer =  randint(0,2)
    if integer == 0:
        flipped_img = cv2.flip(image, 0)
    else:
        flipped_img = cv2.flip(image, 1)
    
    cv2.imwrite(flippeddir + "/" + str(i) + "-" + file,flipped_img)

In [3]:
def add_noise(origdir,noisedir,file,i):
    image = cv2.imread(origdir + "/" + file,0)
    sigma = randint(20, 50)
    
    temp_image = np.float64(np.copy(image))
    h = temp_image.shape[0]
    w = temp_image.shape[1]
    noise = np.random.randn(h, w) * sigma

    noisy_image = np.zeros(temp_image.shape, np.float64)
    if len(temp_image.shape) == 2:
        noisy_image = temp_image + noise
    else:
        noisy_image[:,:,0] = temp_image[:,:,0] + noise
        noisy_image[:,:,1] = temp_image[:,:,1] + noise
        noisy_image[:,:,2] = temp_image[:,:,2] + noise
        
    cv2.imwrite(noisedir + "/" + str(i) + "-" + file,noisy_image)

In [4]:
def random_rotate(origdir,randomrotatedir,file,i):
    image = cv2.imread(origdir + "/" + file,0)
    angle =  randint(-10,10)
    rotated = imutils.rotate_bound(image, angle)
    cv2.imwrite(randomrotatedir + "/" + str(i) + "-" + file,rotated)

In [5]:
def scale_in(origdir,scaleindir,file,i):
    image = cv2.imread(origdir + "/" + file,0)
    #randomized the scaling
    scale = random.uniform(0.1, 0.9)
    height, width = image.shape
    scale_out = transform.rescale(image, scale=scale, mode='constant')
    #rescale Image after scale in
    resized_image = cv2.resize(scale_out, (width, height))
    cv2.imwrite(scaleindir + "/" + str(i) + "-" + file,resized_image)

In [6]:
def scale_out(origdir,scaleoutdir,file,i):
    image = cv2.imread(origdir + "/" + file,0)
    #randomized the scaling
    scale = random.uniform(1.1, 2.2)
    height, width = image.shape
    scale_out = transform.rescale(image, scale=scale, mode='constant')
    #rescale Image after scale in
    resized_image = cv2.resize(scale_out, (width, height))
    cv2.imwrite(scaleoutdir + "/" + str(i) + "-" + file,resized_image)

In [11]:
# Augment Data
augment_dir = "/home/james/Kaggle/Whale/Data/train/augment"
origdir = "/home/james/Kaggle/Whale/Data/train/augment/1"
combineddir = "/home/james/Kaggle/Whale/Data/train/augment/combined"
validdir = "/home/james/Kaggle/Whale/Data/train/augment/valid"

In [12]:
filenames = os.listdir(origdir)
filenames.sort()

In [9]:
#iterations = 9
##Augment the data
#for i in range(iterations):
#    for filename in filenames:
#        seed = randint(1,10)
#        if seed == 1:
#            # Just copy the image
#            copyfile(origdir + "/" + filename, combineddir + "/" + str(i) + "-"+ filename)
#        #These are more valuable, so do more of them
#        elif seed == 2 or seed == 7 or seed == 8:
#            #Add Random Noise to image
##            add_noise(origdir,combineddir,filename,i)
#        elif seed == 3:
#            #Randomly rotate image
#            random_rotate(origdir,combineddir,filename,i)
#        elif seed == 4 or seed == 9:
#            #Randomly Scale in and resize image
#            scale_in(origdir,combineddir,filename,i)
#        elif seed == 5 or seed == 10:
#            #Randomly Scale in and resize image
#            scale_out(origdir,combineddir,filename,i)
#        elif seed == 6:
#            #Randomly flip vertical/horizontal
#            flip_image(origdir,combineddir,filename,i)

In [13]:
iterations = 2
#Create some validation data
for i in range(iterations):
    for filename in filenames:
        seed = randint(1,10)
        #These are more valuable, so do more of them
        if seed == 2 or seed == 7 or seed == 8:
            #Add Random Noise to image
            add_noise(origdir,validdir,filename,i)
        elif seed == 3 or seed == 1:
            #Randomly rotate image
            random_rotate(origdir,validdir,filename,i)
        elif seed == 4 or seed == 9:
            #Randomly Scale in and resize image
            scale_in(origdir,validdir,filename,i)
        elif seed == 5 or seed == 10:
            #Randomly Scale in and resize image
            scale_out(origdir,validdir,filename,i)
        elif seed == 6:
            #Randomly flip vertical/horizontal
            flip_image(origdir,validdir,filename,i)